In [1]:
from glob import glob
import os

basePath = 'highway/valid/label/'

valid_filename = []
for filepath in glob(os.path.join(basePath, '*.xml')):
    filename = os.path.basename(filepath)  ## 상위폴더 경로 제거
    filename = filename[:-4]  ## .xml 제거
    valid_filename.append(filename)

len(valid_filename)

51

In [2]:
valid_filename[1]

'Suwon_CH01_20200721_1700_TUE_9m_RH_highway_TW5_sunny_FHD'

In [3]:
import pandas as pd

df = pd.DataFrame(valid_filename)
df.columns = ['valid_folder']
df.head()

,valid_folder
0,Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5...
1,Suwon_CH01_20200721_1700_TUE_9m_RH_highway_TW5...
2,Suwon_CH01_20200722_1430_WED_9m_NH_highway_TW5...
3,Suwon_CH01_20200722_1930_WED_9m_NH_highway_TW5...
4,Suwon_CH01_20201012_1723_MON_9m_RH_highway_TW5...


In [4]:
import numpy as np


def glob_files(path, file_type="*"):
    search_string = os.path.join(path, file_type)
    files = glob(search_string)

    # print('searching ', path)
    paths = []
    for f in files:
      if os.path.isdir(f):
        sub_paths = glob_files(f + '/')
        paths += sub_paths
      else:
        paths.append(f)

    # We sort the images in alphabetical order to match them
    #  to the annotation files
    paths.sort()

    return paths


WIDTH = 1
HEIGHT = 1

def load_labels(path):
  files = glob_files(path, "*.txt")

  Y_data = []
  for file in files:
    with open(file) as f:
      lines = f.readlines()

      boxes = []
      for line in lines:
        tokens = line.split()

        class_id = int(tokens[0])
        xc = float(tokens[1]) * WIDTH
        yc = float(tokens[2]) * HEIGHT
        width = float(tokens[3]) * WIDTH
        height = float(tokens[4]) * HEIGHT

        boxes.append(np.array([class_id, xc, yc, width, height]))
        # print(class_id, xc, yc, width, height)

      Y_data.append(np.array(boxes))
      # print(lines)
  return Y_data  ## 이미지마다 탐지 대상 개수가 달라서(행 개수가 달라서) 정해진 shape의 넘파이 어레이로 변환할 수 없다! 변환하려면 패딩 등이 필요.

Y_test_ex = load_labels(os.path.join(basePath, 'Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD'))

In [5]:
len(Y_test_ex)

150

In [6]:
Y_test_ex[0]

array([[2.     , 0.59856, 0.27826, 0.02933, 0.01553],
       [0.     , 0.71264, 0.81044, 0.23158, 0.1062 ],
       [0.     , 0.9334 , 0.38367, 0.05759, 0.0243 ],
       [0.     , 0.27496, 0.4214 , 0.07786, 0.03671],
       [0.     , 0.38063, 0.39466, 0.06705, 0.03163],
       [0.     , 0.362  , 0.51417, 0.11412, 0.0466 ],
       [2.     , 0.12581, 0.31099, 0.06983, 0.03639],
       [0.     , 0.04041, 0.34171, 0.04828, 0.01639],
       [1.     , 0.27924, 0.30045, 0.05908, 0.03639],
       [2.     , 0.30308, 0.28079, 0.04191, 0.02792],
       [2.     , 0.53884, 0.27182, 0.04828, 0.03519],
       [2.     , 0.74996, 0.22923, 0.04556, 0.02401],
       [1.     , 0.44876, 0.2657 , 0.0545 , 0.02784],
       [1.     , 0.78748, 0.23394, 0.04635, 0.0249 ],
       [0.     , 0.89222, 0.39231, 0.06023, 0.02519],
       [0.     , 0.32338, 0.32854, 0.04144, 0.01584],
       [0.     , 0.37065, 0.34126, 0.04961, 0.02178],
       [0.     , 0.22014, 0.32191, 0.0367 , 0.01661],
       [0.     , 0.20333, 0.

폴더 하나에 대해서 150개 txt 파일 -> txt 파일명 뽑아오는 법은...?

In [7]:
folderPath_ex = os.path.join(basePath, 'Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD')
txtfilenameList = glob(os.path.join(folderPath_ex, '*.txt'))
txtfilenameList

['highway/valid/label/Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD\\Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD_001.txt',
 'highway/valid/label/Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD\\Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD_002.txt',
 'highway/valid/label/Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD\\Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD_003.txt',
 'highway/valid/label/Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD\\Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD_004.txt',
 'highway/valid/label/Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD\\Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD_005.txt',
 'highway/valid/label/Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD\\Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD_006.txt',
 'highway/valid/label/Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD\\Suwon_CH01_20200720_1830_MON_9m_RH_hig

In [8]:
len(txtfilenameList)

150

In [9]:
os.path.basename(txtfilenameList[0])

'Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD_001.txt'

In [10]:
txt_df = pd.DataFrame(txtfilenameList)
txt_df.columns = ['txt_in_valid_folder']
txt_df

,txt_in_valid_folder
0,highway/valid/label/Suwon_CH01_20200720_1830_M...
1,highway/valid/label/Suwon_CH01_20200720_1830_M...
2,highway/valid/label/Suwon_CH01_20200720_1830_M...
3,highway/valid/label/Suwon_CH01_20200720_1830_M...
4,highway/valid/label/Suwon_CH01_20200720_1830_M...
...,...
145,highway/valid/label/Suwon_CH01_20200720_1830_M...
146,highway/valid/label/Suwon_CH01_20200720_1830_M...
147,highway/valid/label/Suwon_CH01_20200720_1830_M...
148,highway/valid/label/Suwon_CH01_20200720_1830_M...


In [11]:
df_ex = pd.DataFrame(pd.DataFrame(Y_test_ex[0])[0].value_counts()).T
df_ex

,0.0,2.0,1.0
count,21,8,3


In [12]:
df_ex.loc['count', 0.0]

21

In [13]:
txt_df.loc[0, 'car_cnt'] = df_ex.loc['count', 0.0]
txt_df.head()

,txt_in_valid_folder,car_cnt
0,highway/valid/label/Suwon_CH01_20200720_1830_M...,21.0
1,highway/valid/label/Suwon_CH01_20200720_1830_M...,NaN
2,highway/valid/label/Suwon_CH01_20200720_1830_M...,NaN
3,highway/valid/label/Suwon_CH01_20200720_1830_M...,NaN
4,highway/valid/label/Suwon_CH01_20200720_1830_M...,NaN


In [14]:
classes = ['car', 'bus', 'truck']

In [15]:
for i in range(3):
    # print(float(i))
    col = classes[i] + '_cnt'
    # print(col)
    txt_df.loc[0, col] = df_ex.loc['count', float(i)]

In [16]:
txt_df.head()

,txt_in_valid_folder,car_cnt,bus_cnt,truck_cnt
0,highway/valid/label/Suwon_CH01_20200720_1830_M...,21.0,3.0,8.0
1,highway/valid/label/Suwon_CH01_20200720_1830_M...,NaN,NaN,NaN
2,highway/valid/label/Suwon_CH01_20200720_1830_M...,NaN,NaN,NaN
3,highway/valid/label/Suwon_CH01_20200720_1830_M...,NaN,NaN,NaN
4,highway/valid/label/Suwon_CH01_20200720_1830_M...,NaN,NaN,NaN


In [17]:
for i in range(10):
    df_ex = pd.DataFrame(pd.DataFrame(Y_test_ex[i])[0].value_counts()).T
    for j in range(3):
        # print(float(i))
        col = classes[j] + '_cnt'
        # print(col)
        txt_df.loc[i, col] = df_ex.loc['count', float(j)]

In [18]:
txt_df.head(15)

,txt_in_valid_folder,car_cnt,bus_cnt,truck_cnt
0,highway/valid/label/Suwon_CH01_20200720_1830_M...,21.0,3.0,8.0
1,highway/valid/label/Suwon_CH01_20200720_1830_M...,15.0,4.0,7.0
2,highway/valid/label/Suwon_CH01_20200720_1830_M...,5.0,1.0,9.0
3,highway/valid/label/Suwon_CH01_20200720_1830_M...,8.0,1.0,7.0
4,highway/valid/label/Suwon_CH01_20200720_1830_M...,16.0,2.0,4.0
5,highway/valid/label/Suwon_CH01_20200720_1830_M...,9.0,1.0,4.0
6,highway/valid/label/Suwon_CH01_20200720_1830_M...,9.0,1.0,4.0
7,highway/valid/label/Suwon_CH01_20200720_1830_M...,7.0,3.0,4.0
8,highway/valid/label/Suwon_CH01_20200720_1830_M...,19.0,5.0,6.0
9,highway/valid/label/Suwon_CH01_20200720_1830_M...,16.0,4.0,9.0


## 한 폴더 대상 총정리

In [19]:
len(txt_df)

150

In [20]:
for i in range(len(txt_df)):
    df_ex = pd.DataFrame(pd.DataFrame(Y_test_ex[i])[0].value_counts()).T
    for cls in df_ex.columns:
        # print(float(i))
        col = classes[int(cls)] + '_cnt'
        # print(col)
        txt_df.loc[i, col] = df_ex.loc['count', cls]

In [21]:
txt_df.dropna(inplace=True)
txt_df

,txt_in_valid_folder,car_cnt,bus_cnt,truck_cnt
0,highway/valid/label/Suwon_CH01_20200720_1830_M...,21.0,3.0,8.0
1,highway/valid/label/Suwon_CH01_20200720_1830_M...,15.0,4.0,7.0
2,highway/valid/label/Suwon_CH01_20200720_1830_M...,5.0,1.0,9.0
3,highway/valid/label/Suwon_CH01_20200720_1830_M...,8.0,1.0,7.0
4,highway/valid/label/Suwon_CH01_20200720_1830_M...,16.0,2.0,4.0
...,...,...,...,...
144,highway/valid/label/Suwon_CH01_20200720_1830_M...,27.0,1.0,12.0
145,highway/valid/label/Suwon_CH01_20200720_1830_M...,23.0,2.0,12.0
146,highway/valid/label/Suwon_CH01_20200720_1830_M...,20.0,1.0,8.0
147,highway/valid/label/Suwon_CH01_20200720_1830_M...,23.0,1.0,12.0


In [22]:
test_df = pd.DataFrame()

In [25]:
pd.concat([test_df, txt_df])

,txt_in_valid_folder,car_cnt,bus_cnt,truck_cnt
0,highway/valid/label/Suwon_CH01_20200720_1830_M...,21.0,3.0,8.0
1,highway/valid/label/Suwon_CH01_20200720_1830_M...,15.0,4.0,7.0
2,highway/valid/label/Suwon_CH01_20200720_1830_M...,5.0,1.0,9.0
3,highway/valid/label/Suwon_CH01_20200720_1830_M...,8.0,1.0,7.0
4,highway/valid/label/Suwon_CH01_20200720_1830_M...,16.0,2.0,4.0
...,...,...,...,...
144,highway/valid/label/Suwon_CH01_20200720_1830_M...,27.0,1.0,12.0
145,highway/valid/label/Suwon_CH01_20200720_1830_M...,23.0,2.0,12.0
146,highway/valid/label/Suwon_CH01_20200720_1830_M...,20.0,1.0,8.0
147,highway/valid/label/Suwon_CH01_20200720_1830_M...,23.0,1.0,12.0


# 전체 폴더 대상 총정리(미완성)

In [1]:
from glob import glob
import os
import numpy as np
import pandas as pd


basePath = 'highway/valid/label/'

valid_filename = []
for filepath in glob(os.path.join(basePath, '*.xml')):
    filename = os.path.basename(filepath)  ## 상위폴더 경로 제거
    filename = filename[:-4]  ## .xml 제거
    valid_filename.append(filename)



def glob_files(path, file_type="*"):
    search_string = os.path.join(path, file_type)
    files = glob(search_string)

    # print('searching ', path)
    paths = []
    for f in files:
      if os.path.isdir(f):
        sub_paths = glob_files(f + '/')
        paths += sub_paths
      else:
        paths.append(f)

    # We sort the images in alphabetical order to match them
    #  to the annotation files
    paths.sort()

    return paths


WIDTH = 1
HEIGHT = 1

def load_labels(path):
  files = glob_files(path, "*.txt")

  Y_data = []
  for file in files:
    with open(file) as f:
      lines = f.readlines()

      boxes = []
      for line in lines:
        tokens = line.split()

        class_id = int(tokens[0])
        xc = float(tokens[1]) * WIDTH
        yc = float(tokens[2]) * HEIGHT
        width = float(tokens[3]) * WIDTH
        height = float(tokens[4]) * HEIGHT

        boxes.append(np.array([class_id, xc, yc, width, height]))
        # print(class_id, xc, yc, width, height)

      Y_data.append(np.array(boxes))
      # print(lines)
  return Y_data  ## 이미지마다 탐지 대상 개수가 달라서(행 개수가 달라서) 정해진 shape의 넘파이 어레이로 변환할 수 없다! 변환하려면 패딩 등이 필요.



classes = ['car', 'bus', 'truck']
valid_total_df = pd.DataFrame()

for foldername in valid_filename:
    Y_test = load_labels(os.path.join(basePath, foldername))
    folderPath = os.path.join(basePath, foldername)
    txtfilenameList = glob(os.path.join(folderPath, '*.txt'))
    txt_df = pd.DataFrame(txtfilenameList)
    txt_df.columns = ['txt_in_valid_folder']
    for i in range(len(txt_df)):
        df_ex = pd.DataFrame(pd.DataFrame(Y_test[i])[0].value_counts()).T
        for cls in df_ex.columns:
            # print(float(i))
            col = classes[int(cls)] + '_cnt'
            # print(col)
            txt_df.loc[i, col] = df_ex.loc['count', cls]
    # txt_df.dropna(inplace=True)
    valid_total_df = pd.concat([valid_total_df, txt_df])

valid_total_df.reset_index(drop=True, inplace=True)
valid_total_df

,txt_in_valid_folder,car_cnt,truck_cnt,bus_cnt
0,highway/valid/label/Suwon_CH01_20200720_1830_M...,21.0,8.0,3.0
1,highway/valid/label/Suwon_CH01_20200720_1830_M...,15.0,7.0,4.0
2,highway/valid/label/Suwon_CH01_20200720_1830_M...,5.0,9.0,1.0
3,highway/valid/label/Suwon_CH01_20200720_1830_M...,8.0,7.0,1.0
4,highway/valid/label/Suwon_CH01_20200720_1830_M...,16.0,4.0,2.0
...,...,...,...,...
219,highway/valid/label/Suwon_CH10_20201213_0752_S...,2.0,2.0,NaN
220,highway/valid/label/Suwon_CH10_20201213_0752_S...,NaN,1.0,NaN
221,highway/valid/label/Suwon_CH10_20201213_0752_S...,3.0,2.0,NaN
222,highway/valid/label/Suwon_CH10_20201213_0752_S...,2.0,2.0,NaN


In [2]:
valid_total_df.reset_index(drop=True, inplace=True)
valid_total_df

,txt_in_valid_folder,car_cnt,truck_cnt,bus_cnt
0,highway/valid/label/Suwon_CH01_20200720_1830_M...,21.0,8.0,3.0
1,highway/valid/label/Suwon_CH01_20200720_1830_M...,15.0,7.0,4.0
2,highway/valid/label/Suwon_CH01_20200720_1830_M...,5.0,9.0,1.0
3,highway/valid/label/Suwon_CH01_20200720_1830_M...,8.0,7.0,1.0
4,highway/valid/label/Suwon_CH01_20200720_1830_M...,16.0,4.0,2.0
...,...,...,...,...
9329,highway/valid/label/Suwon_CH10_20201213_0752_S...,2.0,2.0,NaN
9330,highway/valid/label/Suwon_CH10_20201213_0752_S...,NaN,1.0,NaN
9331,highway/valid/label/Suwon_CH10_20201213_0752_S...,3.0,2.0,NaN
9332,highway/valid/label/Suwon_CH10_20201213_0752_S...,2.0,2.0,NaN


In [3]:
valid_total_df.to_csv('valid_total_txt_label.csv', index=False)

In [2]:
import pandas as pd

valid_total_df = pd.read_csv('valid_total_txt_label.csv')
valid_total_df.head()

,txt_in_valid_folder,car_cnt,truck_cnt,bus_cnt
0,highway/valid/label/Suwon_CH01_20200720_1830_M...,21.0,8.0,3.0
1,highway/valid/label/Suwon_CH01_20200720_1830_M...,15.0,7.0,4.0
2,highway/valid/label/Suwon_CH01_20200720_1830_M...,5.0,9.0,1.0
3,highway/valid/label/Suwon_CH01_20200720_1830_M...,8.0,7.0,1.0
4,highway/valid/label/Suwon_CH01_20200720_1830_M...,16.0,4.0,2.0


In [4]:
condition1 = valid_total_df['car_cnt'].isnull()
condition2 = valid_total_df['truck_cnt'].isnull()
valid_total_df[(condition1) & (condition2)]

,txt_in_valid_folder,car_cnt,truck_cnt,bus_cnt
3399,highway/valid/label/Suwon_CH05_20200720_1830_M...,NaN,NaN,2.0
3418,highway/valid/label/Suwon_CH05_20200720_1830_M...,NaN,NaN,2.0
4316,highway/valid/label/Suwon_CH06_20200722_1900_W...,NaN,NaN,1.0
9136,highway/valid/label/Suwon_CH10_20201213_0752_S...,NaN,NaN,2.0
9176,highway/valid/label/Suwon_CH10_20201213_0752_S...,NaN,NaN,1.0


트레이닝에서 이런 파일만 찾아서 넣어주면 된다!!

In [7]:
valid_total_df[(condition1) & (condition2)]['txt_in_valid_folder'].values

array(['highway/valid/label/Suwon_CH05_20200720_1830_MON_9m_NH_highway_OW5_sunny_FHD\\Suwon_CH05_20200720_1830_MON_9m_NH_highway_OW5_sunny_FHD_066.txt',
       'highway/valid/label/Suwon_CH05_20200720_1830_MON_9m_NH_highway_OW5_sunny_FHD\\Suwon_CH05_20200720_1830_MON_9m_NH_highway_OW5_sunny_FHD_085.txt',
       'highway/valid/label/Suwon_CH06_20200722_1900_WED_9m_NH_highway_OW5_sunny_FHD\\Suwon_CH06_20200722_1900_WED_9m_NH_highway_OW5_sunny_FHD_033.txt',
       'highway/valid/label/Suwon_CH10_20201213_0752_SUN_9m_NH_highway_OW5_snow_FHD\\Suwon_CH10_20201213_0752_SUN_9m_NH_highway_OW5_snow_FHD_041.txt',
       'highway/valid/label/Suwon_CH10_20201213_0752_SUN_9m_NH_highway_OW5_snow_FHD\\Suwon_CH10_20201213_0752_SUN_9m_NH_highway_OW5_snow_FHD_082.txt'],
      dtype=object)

In [8]:
busonlyList = list(valid_total_df[(condition1) & (condition2)]['txt_in_valid_folder'].values)

In [10]:
import os

for i in busonlyList:
    basename = os.path.basename(i)
    print(basename)

Suwon_CH05_20200720_1830_MON_9m_NH_highway_OW5_sunny_FHD_066.txt
Suwon_CH05_20200720_1830_MON_9m_NH_highway_OW5_sunny_FHD_085.txt
Suwon_CH06_20200722_1900_WED_9m_NH_highway_OW5_sunny_FHD_033.txt
Suwon_CH10_20201213_0752_SUN_9m_NH_highway_OW5_snow_FHD_041.txt
Suwon_CH10_20201213_0752_SUN_9m_NH_highway_OW5_snow_FHD_082.txt


이 중에서도 보면 snow는 쓸모 없음. sunny만 augmentation 들어가야 할 듯.